## Default prediction from customer spending habit

1. Create base functions for the project.
2. Initial Feature Engineering(Brainstorming)
3. Fit the model to the new features and test the results

In [84]:
# import the relevant computational modules

# data manipulation
import pandas as pd #data processing
import numpy as np #linear algebra

# Models Packages
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Gradient Boosting
import lightgbm as lgb
from sklearn.cross_validation import KFold

### Data Exploration and Feature Engineering --- manipulate and extract features from each table

In [2]:
transaction_df = pd.read_csv('../raw_data/transactions_test.csv')
payment_df = pd.read_csv('../raw_data/paiements_test.csv')
billing_df = pd.read_csv('../raw_data/facturation_test.csv')
performance_df = pd.read_csv('../raw_data/performance_test.csv')

### Transaction table

In [3]:
transaction_df.head()

,ID_CPTE,MERCHANT_CATEGORY_XCD,MERCHANT_CITY_NAME,MERCHANT_COUNTRY_XCD,DECISION_XCD,PRIOR_CREDIT_LIMIT_AMT,TRANSACTION_AMT,TRANSACTION_CATEGORY_XCD,TRANSACTION_DTTM,TRANSACTION_TYPE_XCD,SICGROUP
0,71424379,DD,212619,DP,C,1415.0,55.64,E,2014-08-27 12:00:00,F,AC
1,71424379,DD,212619,DP,C,1518.0,109.20,E,2014-08-24 12:00:00,F,AC
2,71424379,CC,2830838,DP,C,2516.0,65.10,E,2014-12-22 12:00:00,F,AP
3,71424379,CC,1252598,DP,C,2332.0,70.38,E,2014-02-16 12:00:00,F,AP
4,71424379,CC,1063854,DP,C,2430.0,138.03,E,2014-12-23 12:00:00,F,AM


In [4]:
# the type of columns
transaction_df.dtypes

ID_CPTE                       int64
MERCHANT_CATEGORY_XCD        object
MERCHANT_CITY_NAME            int64
MERCHANT_COUNTRY_XCD         object
DECISION_XCD                 object
PRIOR_CREDIT_LIMIT_AMT      float64
TRANSACTION_AMT             float64
TRANSACTION_CATEGORY_XCD     object
TRANSACTION_DTTM             object
TRANSACTION_TYPE_XCD         object
SICGROUP                     object
dtype: object

In [5]:
# convert and select categorical columnst
categorical_columns = ['MERCHANT_CATEGORY_XCD', 'MERCHANT_CITY_NAME', 'MERCHANT_COUNTRY_XCD', 'DECISION_XCD', 
                       'TRANSACTION_CATEGORY_XCD', 'TRANSACTION_TYPE_XCD', 'SICGROUP'] 

In [6]:
# encode the categorical features
lbl = preprocessing.LabelEncoder()
for col in categorical_columns:
    transaction_df[col].fillna('unknown')
    transaction_df[col] = lbl.fit_transform(transaction_df[col].astype(str))

In [7]:
transaction_df.head()

,ID_CPTE,MERCHANT_CATEGORY_XCD,MERCHANT_CITY_NAME,MERCHANT_COUNTRY_XCD,DECISION_XCD,PRIOR_CREDIT_LIMIT_AMT,TRANSACTION_AMT,TRANSACTION_CATEGORY_XCD,TRANSACTION_DTTM,TRANSACTION_TYPE_XCD,SICGROUP
0,71424379,15,2779,66,2,1415.0,55.64,4,2014-08-27 12:00:00,5,2
1,71424379,15,2779,66,2,1518.0,109.20,4,2014-08-24 12:00:00,5,2
2,71424379,13,4493,66,2,2516.0,65.10,4,2014-12-22 12:00:00,5,15
3,71424379,13,621,66,2,2332.0,70.38,4,2014-02-16 12:00:00,5,15
4,71424379,13,160,66,2,2430.0,138.03,4,2014-12-23 12:00:00,5,12


In [8]:
len(set(transaction_df['ID_CPTE']))

1596

In [9]:
transaction_df[transaction_df['ID_CPTE'] == 99690111].head(10)

,ID_CPTE,MERCHANT_CATEGORY_XCD,MERCHANT_CITY_NAME,MERCHANT_COUNTRY_XCD,DECISION_XCD,PRIOR_CREDIT_LIMIT_AMT,TRANSACTION_AMT,TRANSACTION_CATEGORY_XCD,TRANSACTION_DTTM,TRANSACTION_TYPE_XCD,SICGROUP


In [10]:
### Extract features from transaction_df
# for MERCHANT_CATEGORY_XCD
# sum the total spent money on the merchant category for the individual customer
tmp = transaction_df.groupby(['ID_CPTE', 'MERCHANT_CATEGORY_XCD'])['TRANSACTION_AMT'].sum()
tmp.head()

ID_CPTE   MERCHANT_CATEGORY_XCD
10074849  2                         64.00
          5                        700.73
          6                         21.40
          8                        196.08
          11                       326.31
Name: TRANSACTION_AMT, dtype: float64

In [11]:
tmp = tmp.reset_index()

In [12]:
tmp.head()

,ID_CPTE,MERCHANT_CATEGORY_XCD,TRANSACTION_AMT
0,10074849,2,64.00
1,10074849,5,700.73
2,10074849,6,21.40
3,10074849,8,196.08
4,10074849,11,326.31


In [13]:
tmp = tmp.pivot_table('TRANSACTION_AMT', ['ID_CPTE'], 'MERCHANT_CATEGORY_XCD')
tmp.columns = ['MERCHANT_CATEGORY_' + str(i) for i in tmp.columns]

In [14]:
tmp = tmp.fillna(0)

In [15]:
tmp.head()

,MERCHANT_CATEGORY_0,MERCHANT_CATEGORY_1,MERCHANT_CATEGORY_2,MERCHANT_CATEGORY_3,MERCHANT_CATEGORY_4,MERCHANT_CATEGORY_5,MERCHANT_CATEGORY_6,MERCHANT_CATEGORY_7,MERCHANT_CATEGORY_8,MERCHANT_CATEGORY_9,...,MERCHANT_CATEGORY_45,MERCHANT_CATEGORY_46,MERCHANT_CATEGORY_47,MERCHANT_CATEGORY_48,MERCHANT_CATEGORY_49,MERCHANT_CATEGORY_50,MERCHANT_CATEGORY_51,MERCHANT_CATEGORY_52,MERCHANT_CATEGORY_53,MERCHANT_CATEGORY_54
ID_CPTE,,,,,,,,,,,,,,,,,,,,,
10074849,0.0,0.0,64.0,0.0,0.0,700.73,21.4,0.00,196.08,0.0,...,0.00,0.0,0.0,0.0,0.0,46.70,737.07,0.0,0.0,0.0
10086539,0.0,0.0,0.0,0.0,0.0,0.00,58.3,46.25,0.00,0.0,...,48.09,0.0,0.0,0.0,0.0,0.00,48.70,0.0,0.0,0.0
10140908,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,...,88.89,0.0,0.0,0.0,0.0,92.02,133.26,0.0,0.0,0.0
10153124,0.0,0.0,0.0,0.0,0.0,123.00,0.0,0.00,0.00,0.0,...,0.00,0.0,28.0,0.0,0.0,0.00,221.98,0.0,0.0,0.0
10211370,0.0,0.0,71.8,0.0,0.0,694.08,0.0,0.00,0.00,0.0,...,1390.48,0.0,0.0,0.0,0.0,21.56,435.75,0.0,0.0,0.0


### Payment table

In [17]:
payment_df.head()

,ID_CPTE,TRANSACTION_AMT,TRANSACTION_DTTM,PAYMENT_REVERSAL_XFLG
0,71424379,686.00,2014-12-14 00:00:00,Q
1,71424379,680.74,2014-03-26 04:00:00,Q
2,71424379,1116.96,2014-08-24 00:00:00,Q
3,71424379,808.78,2014-05-27 00:00:00,Q
4,71424379,949.20,2014-09-15 00:00:00,Q


In [18]:
len(set(payment_df['ID_CPTE']))

5100

In [19]:
# null / total
payment_df['TRANSACTION_AMT'].isnull().sum() / len(payment_df)

0.0003314416905143005

In [20]:
# drop the null rows
payment_df = payment_df.dropna()

In [21]:
# get the month and year of TRANSACTION_DTTM
payment_df['TRANSACTION_DTTM'] = payment_df['TRANSACTION_DTTM'].apply(lambda x: str(x).split(' ')[0][:-3])

In [22]:
payment_df = payment_df.sort_values(['ID_CPTE', 'TRANSACTION_DTTM'])

In [23]:
payment_df = payment_df.groupby(['ID_CPTE', 'TRANSACTION_DTTM'])['TRANSACTION_AMT'].sum().reset_index()

In [24]:
payment_df = payment_df.groupby('ID_CPTE').tail(12)

In [25]:
payment_df['TRANSACTION_DTTM'] = payment_df['TRANSACTION_DTTM'].apply(lambda x: x.split('-')[1])

In [26]:
payment_transaction = payment_df.pivot_table('TRANSACTION_AMT', ['ID_CPTE'], 'TRANSACTION_DTTM')
payment_transaction.columns = ['transaction_' + str(i) for i in payment_transaction.columns + '_month']

In [27]:
payment_transaction = payment_transaction.reset_index()

In [28]:
payment_transaction = payment_transaction.fillna(0)

### Billing table

In [29]:
billing_df.head()

,ID_CPTE,PERIODID_MY,StatementDate,CurrentTotalBalance,CashBalance,CreditLimit,DelqCycle
0,71424379,2013-11-01,2013-11-04,1444.30,0.0,3200.0,0
1,71424379,2014-05-01,2014-04-30,785.89,0.0,3200.0,0
2,71424379,2014-08-01,2014-08-02,1095.48,0.0,3200.0,0
3,71424379,2014-04-01,2014-04-02,845.30,0.0,3200.0,0
4,71424379,2013-12-01,2013-11-30,1623.28,0.0,3200.0,0


In [30]:
len(set(billing_df['ID_CPTE']))

5100

In [31]:
billing_df[billing_df['ID_CPTE'] == 99690111].sort_values(['StatementDate'])

,ID_CPTE,PERIODID_MY,StatementDate,CurrentTotalBalance,CashBalance,CreditLimit,DelqCycle


In [32]:
billing_df['PERIODID_MY'] = billing_df['PERIODID_MY'].apply(lambda x: x[:-3])

In [33]:
billing_df = billing_df.sort_values(['ID_CPTE', 'PERIODID_MY'])
billing_df = billing_df.reset_index(drop=True)

In [34]:
billing_df = billing_df.groupby('ID_CPTE').tail(12)
billing_df = billing_df.reset_index(drop=True)

In [35]:
billing_df['PERIODID_MY'] = billing_df['PERIODID_MY'].apply(lambda x: x[-2:])

In [36]:
# CurrentTotalBalance
total_balance = billing_df.pivot_table('CurrentTotalBalance', ['ID_CPTE'], 'PERIODID_MY')
total_balance.columns = ['total_balance_' + str(i) for i in total_balance.columns + '_month']

In [37]:
# CashBalance
cash_balance = billing_df.pivot_table('CashBalance', ['ID_CPTE'], 'PERIODID_MY')
cash_balance.columns = ['cash_balance_' + str(i) for i in cash_balance.columns + '_month']

In [38]:
# CreditLimit
credit_limit = billing_df.pivot_table('CreditLimit', ['ID_CPTE'], 'PERIODID_MY')
credit_limit.columns = ['credit_limit_' + str(i) for i in credit_limit.columns + '_month']

In [39]:
# DelqCycle
delq_cycle = billing_df.pivot_table('DelqCycle', ['ID_CPTE'], 'PERIODID_MY')
delq_cycle.columns = ['delq_cycle_' + str(i) for i in delq_cycle.columns + '_month']

In [40]:
total_balance = total_balance.reset_index()
cash_balance = cash_balance.reset_index()
credit_limit = credit_limit.reset_index()
delq_cycle = delq_cycle.reset_index()

In [67]:
tmp = total_balance.merge(cash_balance, on='ID_CPTE')
tmp = tmp.merge(credit_limit, on='ID_CPTE')
tmp = tmp.merge(delq_cycle, on='ID_CPTE')

In [68]:
tmp.head()

,ID_CPTE,total_balance_01_month,total_balance_02_month,total_balance_03_month,total_balance_04_month,total_balance_05_month,total_balance_06_month,total_balance_07_month,total_balance_08_month,total_balance_09_month,...,delq_cycle_03_month,delq_cycle_04_month,delq_cycle_05_month,delq_cycle_06_month,delq_cycle_07_month,delq_cycle_08_month,delq_cycle_09_month,delq_cycle_10_month,delq_cycle_11_month,delq_cycle_12_month
0,10074849,43.43,231.00,0.00,617.11,10.40,344.5,185.11,8.24,396.93,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10086539,546.00,772.65,508.00,667.68,988.79,1407.6,290.85,754.80,998.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10140908,3822.04,3926.90,3512.78,3533.25,3212.00,3294.9,2983.90,2873.52,2657.40,...,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0
3,10147994,1990.80,2526.27,2756.32,2710.05,3138.41,3258.9,3895.38,4052.00,4230.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10152808,14756.56,14394.80,13853.16,14499.31,14728.48,14525.0,14343.00,14034.00,14389.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
tmp = payment_transaction.merge(tmp, on='ID_CPTE', how='right')

In [70]:
len(tmp)

5100

In [71]:
transaction_col = tmp.iloc[:, :12].columns

for col in transaction_col:
    tmp[col] = tmp[col].fillna(tmp[tmp[col].notna()][col].mean())

In [72]:
tmp = tmp.fillna(0)
tmp = tmp.sample(frac=1)

In [74]:
tmp = tmp.set_index(['ID_CPTE'])

In [87]:
from sklearn.decomposition import PCA
pca = PCA(n_components=30)
principalComponents = pca.fit_transform(tmp)

In [54]:
from sklearn.linear_model import LogisticRegression

In [55]:
from sklearn.externals import joblib

In [85]:
clf = joblib.load('logistic_model.pkl')

In [88]:
predict = clf.predict(principalComponents)

In [89]:
tmp['Default'] = predict

In [90]:
result = tmp.reset_index()[['ID_CPTE', 'Default']]

In [92]:
result.to_csv('results.csv')